In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_625176/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_20_9_8,0.999937,0.789982,0.999479,0.999658,0.999698,0.000089,0.298090,0.000225,0.000093,0.000159,0.024052,0.009428,1.000006,0.009829,586.656294,932.817028,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
1,model_20_9_9,0.999937,0.790007,0.999397,0.999607,0.999652,0.000090,0.298055,0.000261,0.000106,0.000184,0.024566,0.009463,1.000006,0.009866,586.641411,932.802146,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
2,model_20_9_7,0.999937,0.789954,0.999564,0.999709,0.999746,0.000090,0.298130,0.000189,0.000079,0.000134,0.023483,0.009474,1.000006,0.009877,586.636960,932.797694,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
3,model_20_9_10,0.999936,0.790028,0.999318,0.999558,0.999607,0.000091,0.298025,0.000295,0.000120,0.000207,0.025036,0.009556,1.000006,0.009963,586.602372,932.763107,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
4,model_20_9_6,0.999935,0.789921,0.999648,0.999759,0.999794,0.000093,0.298177,0.000152,0.000065,0.000109,0.022856,0.009627,1.000006,0.010037,586.572584,932.733318,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,model_8_7_7,0.996120,0.808244,0.996614,0.989916,0.996967,0.005507,0.272170,0.005728,0.003919,0.004824,0.609164,0.074210,1.000054,0.077369,3528.403415,5672.405991,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
1238,model_74_6_0,0.996088,0.841538,0.999699,0.998378,0.999309,0.005552,0.224914,0.000435,0.001011,0.000723,0.203028,0.074511,1.000162,0.077683,1216.387257,1951.369379,"Hidden Size=[30, 16], regularizer=0.2, learnin..."
1265,model_72_6_0,0.995942,0.795672,0.996610,0.999949,0.999197,0.005759,0.290015,0.002565,0.000133,0.001349,0.052461,0.075889,1.000422,0.079120,520.313929,831.127264,"Hidden Size=[24, 7], regularizer=0.5, learning..."
1268,model_36_6_2,0.995922,0.791520,0.996879,0.999721,0.999027,0.005788,0.295907,0.003251,0.000895,0.002073,0.106649,0.076081,1.000121,0.079320,1680.303810,2698.065124,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
